In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
import random

#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dense
#from tensorflow.keras.layers import Dropout

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('/content/drive/MyDrive/DataColab/intents.json').read()
intents = json.loads(data_file)

In [ ]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))
        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

In [ ]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [ ]:
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

In [ ]:
random.shuffle(training)

# Extract X_train (features) and y_train (labels) from training
train_x = np.array([sample[0] for sample in training])  # Features
train_y = np.array([sample[1] for sample in training])
print("Training data created")

Training data created


In [ ]:
model = Sequential()
model.add(Dense(256, input_shape=(len(train_x[0]),)))
model.add(Activation('relu'))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0])))
model.add(Activation('softmax'))

In [ ]:
from tensorflow.keras.optimizers import SGD

# Define the SGD optimizer with updated parameters
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)

# Compile the model with the updated optimizer
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# Fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print("Model created")

Epoch 1/200
162/162 [==============================] - 2s 3ms/step - loss: 4.1272 - accuracy: 0.0506
Epoch 2/200
162/162 [==============================] - 1s 3ms/step - loss: 3.9905 - accuracy: 0.0728
Epoch 3/200
162/162 [==============================] - 1s 3ms/step - loss: 3.7575 - accuracy: 0.1247
Epoch 4/200
162/162 [==============================] - 1s 3ms/step - loss: 3.4576 - accuracy: 0.1741
Epoch 5/200
162/162 [==============================] - 1s 3ms/step - loss: 3.1790 - accuracy: 0.2160
Epoch 6/200
162/162 [==============================] - 1s 3ms/step - loss: 2.8785 - accuracy: 0.2790
Epoch 7/200
162/162 [==============================] - 1s 3ms/step - loss: 2.6287 - accuracy: 0.3148
Epoch 8/200
162/162 [==============================] - 1s 3ms/step - loss: 2.4213 - accuracy: 0.3593
Epoch 9/200
162/162 [==============================] - 1s 3ms/step - loss: 2.2508 - accuracy: 0.3815
Epoch 10/200
162/162 [==============================] - 1s 3ms/step - loss: 2.1828 - accura

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
